In [1]:
# https://neo4j.com/docs/neo4j-graphrag-python/current/user_guide_rag.html#using-another-llm-model
# https://neo4j.com/blog/news/graphrag-python-package/
# https://neo4j.com/blog/developer/enhancing-hybrid-retrieval-graphrag-python-package/

# Retrieval Modes and GraphRAG in Neo4j

This section clarifies the different retrieval modes used in GraphRAG systems, how they are combined in practice, and how Neo4j supports them internally through different indexing mechanisms and retrievers.

---

## 1. Three Fundamental Modes of Retrieval

In modern knowledge-assisted LLM systems, there are **three complementary modes of retrieval**:

### 1.1 Vector (Semantic) Search  
This is the retrieval mode used in **classical RAG**.

- Text is embedded into vectors using an embedding model
- Queries are embedded in the same vector space
- Retrieval is based on **semantic similarity**
- Captures meaning, paraphrases, and conceptual similarity

**Strengths:**  
- Works even when exact keywords do not match  
- Robust to paraphrasing  

**Limitations:**  
- Can miss exact terms, names, numbers, or rare tokens  

---

### 1.2 Graph (Query-Based) Search  
This is **symbolic retrieval** over a structured knowledge graph.

- Nodes and relationships represent entities and facts
- Retrieval is done using **Cypher queries**
- Supports joins, constraints, and multi-hop reasoning

**Strengths:**  
- Precise, explainable, deterministic  
- Ideal for structured knowledge and reasoning  

**Limitations:**  
- Requires schema awareness  
- Not suitable for fuzzy or semantic matching  

---

### 1.3 Full-Text (Exact / Lexical) Search  
This is **keyword-based retrieval**, similar to classical information retrieval.

- Operates on raw text
- Uses tokenization and term statistics (e.g. BM25-style scoring)
- Matches exact words or phrases

**Strengths:**  
- Excellent for names, technical terms, identifiers  
- Deterministic and interpretable  

**Limitations:**  
- No semantic understanding  
- Sensitive to wording  

---

## 2. How GraphRAG Combines These Modes

**GraphRAG is not a single retrieval strategy**, but a family of approaches that combine the above modes.

In practice, the most effective GraphRAG pipelines follow this pattern:

> **Retrieve text first, then reason with the graph.**

Typical combinations include:

- **Vector → Graph**  
  Classical RAG retrieval followed by Cypher queries on the graph nodes linked to retrieved text.

- **Vector → Full-Text → Graph**  
  Semantic recall first, lexical filtering second, graph traversal last.

- **Full-Text → Vector → Graph**  
  Exact keyword filtering first, semantic ranking second, graph reasoning last.

The key idea is that **vector and full-text search identify relevant evidence**, while **graph queries provide structure, aggregation, and reasoning**.

---

## 3. How Neo4j Supports These Retrieval Modes

Neo4j supports these retrieval processes **in parallel**, within the same database, using different **indexing mechanisms**.

Conceptually, you can think of Neo4j as hosting:

### 3.1 The Knowledge Graph (Symbolic Layer)
- Nodes: entities (e.g. `Planet`, `Person`, `Company`)
- Relationships: facts and relations
- Accessed via **Cypher queries**

This is the graph used in query-based GraphRAG.

---

### 3.2 The Vector Index (Semantic Layer)
- Built over nodes that store embeddings (typically `Chunk` nodes)
- Enables **k-nearest-neighbor vector search**
- Used for semantic retrieval

---

### 3.3 The Full-Text Index (Lexical Layer)
- Built over text properties of nodes (again, usually `Chunk`)
- Enables keyword-based search
- Complements vector retrieval

> These are **not separate databases**, but **different indexes co-existing on the same Neo4j graph**, each optimized for a different retrieval mode.

---

## 4. Neo4j Retrievers and the Retrieval Modes They Combine

Neo4j provides four retrievers that progressively combine these capabilities.

---

### 4.1 `VectorRetriever`

**Retrieval modes used:**
- Vector search

**Requirements:**
- Vector index

**Description:**  
Performs pure semantic search over embeddings and returns the matched nodes with similarity scores.  
This is the closest analogue to classical RAG retrieval.

---

### 4.2 `VectorCypherRetriever`

**Retrieval modes used:**
- Vector search  
- Graph (Cypher) traversal

**Requirements:**
- Vector index  
- Knowledge graph

**Description:**  
Performs vector search first, then executes a Cypher retrieval query starting from the matched nodes.  
This is often the **first true GraphRAG retriever** users encounter.

---

### 4.3 `HybridRetriever`

**Retrieval modes used:**
- Vector search  
- Full-text search

**Requirements:**
- Vector index  
- Full-text index

**Description:**  
Combines semantic and lexical retrieval to improve recall and precision, but does not yet exploit graph structure.

---

### 4.4 `HybridCypherRetriever`

**Retrieval modes used:**
- Vector search  
- Full-text search  
- Graph (Cypher) traversal

**Requirements:**
- Vector index  
- Full-text index  
- Knowledge graph

**Description:**  
This retriever represents the **full GraphRAG retrieval pipeline**:
semantic + lexical retrieval followed by graph-based contextualization and reasoning.

---

## 5. Summary

- There are **three fundamental retrieval modes**: vector, graph, and full-text
- GraphRAG systems combine these modes in different ways
- Neo4j supports all three via **co-existing indexes on the same graph**
- The four Neo4j retrievers differ in **which modes they combine**
- `HybridCypherRetriever` represents the most complete and production-ready approach

Understanding these layers clarifies why GraphRAG is more than “RAG + a graph” — it is a structured retrieval and reasoning pipeline.


In [2]:
from neo4j import GraphDatabase
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline
from neo4j_graphrag.llm import OllamaLLM
from neo4j_graphrag.embeddings.ollama import OllamaEmbeddings
from neo4j_graphrag.retrievers import VectorRetriever, HybridCypherRetriever
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

In [3]:
# Initialize the ChatOllama model with the specified model name
chat_name = 'gwen2.5-coder:3b'  # Or another text-focused model
cypher_name = 'tomasonjo/llama3-text2cypher-demo:8b_4bit'

# and initialize the ChatOllama instance
chat_model = OllamaLLM(
    model_name=cypher_name,# chat_name,
    model_params={
        "response_format": {"type": "json_object"},
        "temperature": 0.0,
        'format': 'json'
    }
)

In [4]:
embedder_name = 'qwen3-embedding:0.6b'

embedder = OllamaEmbeddings(
    model=embedder_name
)

In [5]:
import os
from dotenv import load_dotenv
from langchain_neo4j import Neo4jGraph

# Load environment variables from .env file
load_dotenv()

# Get credentials from environment variables
neo4j_url = os.getenv("NEO4J_URL", "bolt://localhost:7687")
neo4j_user = os.getenv("NEO4J_USER", "neo4j")
neo4j_password = os.getenv("NEO4J_PASSWORD")

if not neo4j_password:
    raise ValueError("NEO4J_PASSWORD environment variable is not set. Please create a .env file with your credentials.")

graph = Neo4jGraph(
    url=neo4j_url,
    username=neo4j_user,
    password=neo4j_password
)

driver = GraphDatabase.driver(
    neo4j_url,
    auth=(neo4j_user, neo4j_password)
)


In [6]:
text = '''
The solar system consists of the Sun and the objects that orbit it, including planets, moons, asteroids, comets, and meteoroids.
The Sun is a star at the center of the Solar System.
Mercury is a planet in the Solar System. Mercury orbits the Sun. Mercury has no atmosphere and no magnetic field.
Venus is a planet in the Solar System. Venus orbits the Sun. Venus has a thick atmosphere. The atmosphere of Venus is composed mainly of carbon dioxide. Venus has no magnetic field.
Earth is a planet in the Solar System. Earth orbits the Sun. Earth has one moon called the Moon. Earth has a thick atmosphere composed mainly of nitrogen and oxygen. Earth has a strong magnetic field.
Mars is a planet in the Solar System. Mars orbits the Sun. Mars has two moons called Phobos and Deimos. Mars has a thin atmosphere composed mainly of carbon dioxide. Mars has a weak magnetic field.
Jupiter is a planet in the Solar System. Jupiter orbits the Sun. Jupiter has moons called Io, Europa, Ganymede, and Callisto. Jupiter has a thick atmosphere composed mainly of hydrogen and helium. Jupiter has a strong magnetic field.
'''
print(text)


The solar system consists of the Sun and the objects that orbit it, including planets, moons, asteroids, comets, and meteoroids.
The Sun is a star at the center of the Solar System.
Mercury is a planet in the Solar System. Mercury orbits the Sun. Mercury has no atmosphere and no magnetic field.
Venus is a planet in the Solar System. Venus orbits the Sun. Venus has a thick atmosphere. The atmosphere of Venus is composed mainly of carbon dioxide. Venus has no magnetic field.
Earth is a planet in the Solar System. Earth orbits the Sun. Earth has one moon called the Moon. Earth has a thick atmosphere composed mainly of nitrogen and oxygen. Earth has a strong magnetic field.
Mars is a planet in the Solar System. Mars orbits the Sun. Mars has two moons called Phobos and Deimos. Mars has a thin atmosphere composed mainly of carbon dioxide. Mars has a weak magnetic field.
Jupiter is a planet in the Solar System. Jupiter orbits the Sun. Jupiter has moons called Io, Europa, Ganymede, and Callis

## Build the KG

Build the KG and store in a Neo4j database

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from neo4j_graphrag.experimental.components.text_splitters.langchain import LangChainTextSplitterAdapter

In [8]:
kg_pipeline = SimpleKGPipeline(
    driver=driver,
    llm=chat_model,
    embedder=embedder,
    from_pdf=False,
    text_splitter=LangChainTextSplitterAdapter(
        RecursiveCharacterTextSplitter(
            chunk_size=200,
            chunk_overlap=20,
            separators=["\n\n", "\n", " ", ""]
        )
    )
)


In [9]:
await kg_pipeline.run_async(
    text=text
)

content:  {
  "node_types": [
    {
      "label": "SolarSystem",
      "properties": []
    },
    {
      "label": "Star",
      "properties": [
        {
          "name": "name",
          "type": "STRING",
          "required": true
        }
      ]
    },
    {
      "label": "Planet",
      "properties": [
        {
          "name": "name",
          "type": "STRING",
          "required": true
        },
        {
          "name": "atmosphere",
          "type": "STRING",
          "required": false
        },
        {
          "name": "magnetic_field",
          "type": "BOOLEAN",
          "required": false
        }
      ]
    },
    {
      "label": "Moon",
      "properties": [
        {
          "name": "name",
          "type": "STRING",
          "required": true
        }
      ]
    },
    {
      "label": "Asteroid",
      "properties": [
        {
          "name": "name",
          "type": "STRING",
          "required": true
        }
      ]
    },
    {
 

PipelineResult(run_id='a252f0a3-47f6-4f2b-9dc3-b3774bdc5475', result={'resolver': {'number_of_nodes_to_resolve': 31, 'number_of_created_nodes': 13}})

In [10]:
from neo4j_graphrag.indexes import create_vector_index, drop_index_if_exists, create_fulltext_index

# First, drop the existing indexes if they exist
drop_index_if_exists(driver, "text_embeddings")
drop_index_if_exists(driver, "chunk_fulltext")

# Get the embedding dimension by testing with a sample query
sample_embedding = embedder.embed_query("test")
dimensions = len(sample_embedding)
print(f"Embedding dimensions: {dimensions}")

# Create vector index
create_vector_index(
    driver,
    name="text_embeddings",
    label="Chunk",
    embedding_property="embedding",
    dimensions=dimensions,
    similarity_fn="cosine")

# Create full-text index for hybrid search
create_fulltext_index(
    driver,
    name="chunk_fulltext",
    label="Chunk",
    node_properties=["text"])

Embedding dimensions: 1024


In [11]:
# https://neo4j.com/docs/neo4j-graphrag-python/current/api.html#retrievers

In [12]:
vector_retriever = VectorRetriever(
   driver,
   index_name="text_embeddings",
   embedder=embedder
)

In [13]:
import json

vector_res = vector_retriever.get_search_results(
    query_text = "Which planet has a thick atmosphere composed mainly of carbon dioxide?",
    top_k=3)
for i in vector_res.records: print("====n" + json.dumps(i.data(), indent=4))

====n{
    "node": {
        "embedding": null,
        "index": 2,
        "text": "Venus is a planet in the Solar System. Venus orbits the Sun. Venus has a thick atmosphere. The atmosphere of Venus is composed mainly of carbon dioxide. Venus has no magnetic field."
    },
    "nodeLabels": [
        "__KGBuilder__",
        "Chunk"
    ],
    "elementId": "4:ee307052-61bb-4a26-b230-a256204ad709:11",
    "id": "4:ee307052-61bb-4a26-b230-a256204ad709:11",
    "score": 0.8321390151977539
}
====n{
    "node": {
        "embedding": null,
        "index": 2,
        "text": "Venus is a planet in the Solar System. Venus orbits the Sun. Venus has a thick atmosphere. The atmosphere of Venus is composed mainly of carbon dioxide. Venus has no magnetic field."
    },
    "nodeLabels": [
        "__KGBuilder__",
        "Chunk"
    ],
    "elementId": "4:ee307052-61bb-4a26-b230-a256204ad709:37",
    "id": "4:ee307052-61bb-4a26-b230-a256204ad709:37",
    "score": 0.8321390151977539
}
====n{
    "

In [14]:
# and initialize the ChatOllama instance
cypher_model = ChatOllama(
    model=cypher_name,
    validate_model_on_init=True,
    temperature=0
)

In [15]:
cypher_prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are an expert Neo4j Cypher query generator.

TASK:
- Translate the user's natural language question into a SINGLE, valid Cypher query.
- Ensure the query is syntactically correct and follows Cypher rules.

CONSTRAINTS:
- Use ONLY the schema provided below.
- Do NOT invent labels, relationship types, or properties.
- Do NOT explain the query.
- Output ONLY valid Cypher (no additional text, comments, or explanations).
- If the question cannot be answered unambiguously using the schema, output:
  // CANNOT_ANSWER
- Combine all conditions into a single WHERE clause.
- Do not use multiple WHERE clauses or WHERE after RETURN.

GRAPH SCHEMA:
Node labels:
- Star {{name}}
- Planet {{name}}
- Moon {{name}}
- Atmosphere {{description}}
- Substance {{name}}
- PhysicalProperty {{name, value}}

Relationships:
- (Planet)-[:ORBITS]->(Star)
- (Moon)-[:ORBITS]->(Planet)
- (Planet)-[:HAS_ATMOSPHERE]->(Atmosphere)
- (Atmosphere)-[:COMPOSED_OF]->(Substance)
- (Planet)-[:HAS_PROPERTY]->(PhysicalProperty)

QUERY RULES:
1. Always specify node labels.
2. Always specify relationship directions.
3. Use meaningful variable names.
4. Return only properties, not full nodes.
5. Use DISTINCT unless duplicates are required.
6. Use OPTIONAL MATCH if information may be missing.
7. Do not use APOC or procedures.
8. Ensure the query has only one WHERE clause, placed after MATCH clauses.

EXAMPLES:
User: Which planets have atmospheres?
Cypher: MATCH (p:Planet)-[:HAS_ATMOSPHERE]->(a:Atmosphere) RETURN p.name AS planet

User: Which planet has an atmosphere composed of carbon dioxide?
Cypher: MATCH (p:Planet)-[:HAS_ATMOSPHERE]->(a:Atmosphere)-[:COMPOSED_OF]->(s:Substance) WHERE s.name = "carbon dioxide" RETURN p.name AS planet

"""),
    ("human", "{question}")
])

In [18]:
retrieval_query = cypher_model.invoke(
    cypher_prompt.format_messages(
        question="What is Phobos?"
    )
)
print("Generated Cypher Query:")
print(retrieval_query)

Generated Cypher Query:
content='MATCH (m:Moon {name: "Phobos"}) RETURN m.name AS moon' additional_kwargs={} response_metadata={'model': 'tomasonjo/llama3-text2cypher-demo:8b_4bit', 'created_at': '2026-01-28T14:53:03.90516742Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3388124510, 'load_duration': 2166688161, 'prompt_eval_count': 453, 'prompt_eval_duration': 671280493, 'eval_count': 19, 'eval_duration': 512708665, 'logprobs': None, 'model_name': 'tomasonjo/llama3-text2cypher-demo:8b_4bit', 'model_provider': 'ollama'} id='lc_run--019c0517-eee3-7fb0-ae05-2a1f531fdd1f-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 453, 'output_tokens': 19, 'total_tokens': 472}


In [20]:
hybrid_cypher_retriever = HybridCypherRetriever(
    driver=driver,
    vector_index_name="text_embeddings",
    fulltext_index_name="chunk_fulltext",
    retrieval_query=retrieval_query.content,
    embedder=embedder,
)
query_text = "What is Phobos?"
retriever_result = hybrid_cypher_retriever.search(query_text=query_text, top_k=3)
print(retriever_result)

items=[RetrieverResultItem(content="<Record moon='Phobos'>", metadata=None), RetrieverResultItem(content="<Record moon='Phobos'>", metadata=None), RetrieverResultItem(content="<Record moon='Phobos'>", metadata=None)] metadata={'query_vector': [0.015158344, -0.033649556, -0.012692616, -0.0021464932, 0.025392577, -0.0071530994, -0.007510287, 0.03442589, -0.04382649, -0.03620795, -0.043658692, -0.038743947, -0.00027884313, -0.011651735, -0.04599308, 0.097021, 0.016465008, 0.06273234, 0.0777831, -0.051412437, -0.05750918, 0.05983749, -0.004064469, 0.086335406, 0.060814243, 0.027920721, -0.03903848, 0.03047234, -0.013330062, -0.028813072, 0.03672183, 0.016883908, -0.045918997, -0.020436157, -0.018391922, -0.015191395, -0.011909524, -0.048841063, -0.045024157, 0.03970093, 0.028300151, 0.06766281, -0.0010058986, -0.01130612, 0.009167876, 0.02349175, 0.034455366, 0.008783625, 0.04707371, -0.0074459626, -0.031324983, -0.038599685, -0.04006716, -0.036696475, 0.00058767444, -0.04271976, 0.0094691